# Identificação do Aluno
**Nome:** Henrique Shodi Maeta

# Nome do dataset
**Dataset**: [TMDB 5000 Movie Dataset](https://www.kaggle.com/tmdb/tmdb-movie-metadata)

# Resumo do dataset
O dataset possui diversas informações a respeito de 5000 filmes coletados através da Interface de programação de aplicações [The Movie Database API](https://www.themoviedb.org/documentation/api) e pode ser encontrado no [kaggle](https://www.kaggle.com/tmdb/tmdb-movie-metadata).
Apesar de não ser sitado o método de seleção de amostra, acredito que a mesma foi realizada de forma randômica, que é o mais indicado para evitar alguns enviesamentos que poderiam vir a acontecer.

Cada instância de filme, no dataset original, vem com 

Vale ressaltar que, por ser um estudo observacional, não haverá a discriminação de causalidade entre atributos. 

In [ ]:
!pip install plotly
!pip install seaborn --upgrade

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.graph_objects as go

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline

# Questões
1. Há alguma relação entre o orçamento e a receita dos filmes?
  1. Como a avaliação do filme pode impactar no lucro?
2. Quais são os gêneros preferidos por parte das produtoras?
3. Como foi a evolução orçamentária das produtoras durante os anos?
4. O cinema, no geral, é um segmento lucrativo?
5. Qual a relação, caso exista, entre o tamanho da equipe, quantidade de atores e o orçamento da produção?

In [ ]:
credits_data = pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_credits.csv', index_col = 'movie_id')
data = pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_movies.csv', index_col = 'id')

In [ ]:
# join das duas entidades
df = pd.concat([credits_data, data], axis = 1, join = 'inner')

In [ ]:
def get_size(key):
    def size(row):
        return len(json.loads(row[key]))
    return size

def get_genre(row):
    data = json.loads(row['genres'])
    try:
        return data[0]['name']
    except Exception:
        return None
    
df['crew_size'] = df.apply(get_size('crew'), axis = 1)
df['cast_size'] = df.apply(get_size('cast'), axis = 1)
df['lucro'] = df.apply(lambda row: row['revenue'] - row['budget'], axis = 1)
df['genero'] = df.apply(get_genre, axis = 1)
df['release_date'] = pd.to_datetime(df['release_date'], format = '%Y-%m-%d')
df['release_year'] = pd.DatetimeIndex(df['release_date']).year
df = df[df['release_year'].notna()]
df['release_year'] = df['release_year'].astype(int)

## Há alguma relação entre o orçamento e a receita dos filmes?
De acordo com o gráfico abaixo, podemos verificar que existe sim uma relação crescente entre o orçamento e a receita, indicando que os filmes com um maior orçamento tendem a ter uma receita mais elevada, isso pode ocorrer por inúmeros motivos e devem haver algumas variáveis de confusão que devem justificar essa relação, mas é indiscutível que, quanto maior o orçamento, mais ações de divulgação poderão ser realizadas.

É importante notarmos a presença de vários outliers em ambas as distribuições.

In [ ]:
g1 = sns.jointplot(data = df, x = 'budget', y = 'revenue')
g1.fig.set_figheight(10)
g1.fig.set_figwidth(10)
g1.set_axis_labels(xlabel = 'Orçamento', ylabel = 'Receita')
g1.fig.suptitle('Gráfico de disperção para orçamento x receita')

### Como a avaliação do filme pode impactar no lucro?
Podemos ver no gráfico abaixo que existe sim uma relação crescente entre a nota e o lucro, porém o gráfico deixa evidente que mesmo filmes com notas boas podem não apresentar lucro ou até mesmo ter prejuízo.
É importante notar a relação aparenta ser exponencial

In [ ]:
g1 = sns.jointplot(data = df, x = 'vote_average', y = 'lucro')
g1.fig.set_figheight(10)
g1.fig.set_figwidth(10)
g1.set_axis_labels(xlabel = 'Nota média', ylabel = 'Lucro')
g1.fig.suptitle('Gráfico de disperção para lucro x nota')

---
## Quais são os gêneros preferidos por parte das produtoras?
Podemos perceber que há uma clara preferência, por parte dos produtores, pelos estilos de drama, comédia e ação

In [ ]:
plt.figure(figsize=(10,5))
d = df.dropna(axis = 0)
g2 = sns.countplot(data = d, x = 'genero')
plt.xticks(rotation=45)
g2.set(xlabel = 'Gênero', ylabel = 'Quantidade', title = 'Quantidade de filmes produzidos agrupados por gênero')

O boxplot abaixo mostra que, apesar da preferência, a receita mediana não varia muito entre os gêneros, tornando dificil para encontrar, neste dataset, um esboço do motivo pelo qual tais gêneros são tão preferidos pelos produtores.

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.boxplot(data=df, x="genero", y="revenue")
ax.set(ylabel = 'receita ($/1e9)', xlabel = 'gênero', title = 'Receita por gênero')
plt.xticks(rotation=45)

No que tange popularidade dos gêneros, a distribuição aparenta ser bastante uniforme, discartando a hipótese de que a preferência do gênero, citada anteriormente, se dê por esse motivo. 

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.boxplot(data=df, x="genero", y="popularity")
ax.set(ylabel = 'Popularidade', xlabel = 'gênero', title = 'Popularidade por gênero')
plt.xticks(rotation=45)

Podemos ver que o gênero de ação foi o que mais teve crescimento na receita média durante os anos, o que poderia justificar parcialmente a preferência que observamos

In [ ]:
plt.figure(figsize=(10,5))
df_3 = df.groupby([df['release_date'].dt.year, 'genero']).mean()
df_3.reset_index(inplace = True)
df_3 = df_3.loc[(df_3['genero'] == 'Action') | (df_3['genero'] == 'Drama') | (df_3['genero'] == 'Comedy') ]
ax = sns.lineplot(data = df_3, x = 'release_date', y = 'revenue', hue = 'genero')
ax.set(ylabel = 'Receita', xlabel = 'Ano', title = 'Média de receita ao longo dos anos')
ax.legend(title='Gênero', labels=['Drama', 'Ação', 'Comédia'])

<!-- --- -->
## Como foi a evolução orçamentária das produtoras durante os anos?

Podemos ver que o orçamento total teve crescimento expressivo a partir de 1980, muito provavelmente pela popularização dos filmes/cinemas, porém o gráfico mostra uma queda expressiva perto do ano de 2020.

In [ ]:
plt.figure(figsize=(10,5))
df_2 = df.groupby(df['release_date'].dt.year).sum()
df_2.reset_index(inplace = True)
ax = sns.lineplot(data = df_2, x = 'release_date', y = 'budget')
ax.set(ylabel = 'Orçamento', xlabel = 'Ano', title = 'Soma do orçamento dos filmes no ano')

Abaixo, podemos ver que no ano de 2016 e 2017 houveram poucos registros de filmes, provavelmente por conta do método e período em que o dataset foi gerado, justificando a queda expressiva no orçamento citada anteriormente, excluindo os impactos causados pelo COVID-19 que vieram a surgir em meados de 2020. 

In [ ]:
import plotly.express as px
df_release_year = df.groupby('release_year')['release_date'].count()
fig = px.bar(x=df_release_year.index, y=df_release_year)

fig.update_layout(
    title="Quantidade de filmes por ano",
    xaxis_title="Ano",
    yaxis_title="Quantidade de filmes"
)
fig.show()

---
## O cinema, no geral, é um segmento lucrativo?


Podemos ver no gráfico abaixo que o lucro que os produtores possuem em suas produções, em média, o lucro de `$ 2.529.443,00` , porém no histograma fica claro que há muitas ocorrências de lucro negativo.
A média informada foi calculada utilizando-se da mediana por termos uma distribuição com a presença de muitos outliers, caso fosse utilizada a média convencional, poderiamos ter uma informação pouco fidedigna pois outliers possuiem um grande impacto no calculo da mesma. Para ilustrarmos esse impacto, caso fosse utilizado a média convencional, o lucro esperado para um filme qualquer seria de `$ 53.226.680,74`, uma diferença de mais de 50 milhões de dólares, o que não condiz com o histograma abaixo, onde grande parte dos filmes possuem o lucro entre 0 a 50 milhões de dólares.

In [ ]:
fig = px.histogram(df, x="lucro", nbins = 70)
fig.update_layout(
    title="Distribuição do lucro",
    xaxis_title="Lucro",
    yaxis_title=""
)
fig.show()

In [ ]:
df['lucro'].mean()

In [ ]:
df['lucro'].median()

In [ ]:
df['lucro'].describe()

---
## Qual a relação, caso exista, entre o tamanho da equipe, quantidade de atores e o orçamento da produção

O gráfico interativo abaixo representa a quantidade de atores, no eixo `x`, o tamanho da equipe técnica, no eixo `y`, e o orçamento no eixo `z`. Com o gráfico é possível identificarmos uma leve relação crescente entre as variáveis, porém não podemos definir uma relação causal entre elas por ser um dataset observacional.

In [ ]:
import plotly.express as px
genero_not_null = df[df['genero'].notnull()]
fig = px.scatter_3d(genero_not_null, x='cast_size', y='crew_size', color='genero', z='budget', opacity=0.7)
fig.update_layout(scene = dict(
    xaxis_title = 'Quantidade de atores',
    yaxis_title = 'Tamanho da equipe técnica',
    zaxis_title ='Orçamento'))
fig.show()

# Conclusão

Podemos ver que o mercado de filmes veio crescendo expressivamente durante o final do primeiro milênio e meados dos anos 2000 porém ainda nos restaram dúvidas sobre o motivo pela qual há a clara preferência em filmes dos estilos de drama, comédia e ação. Outra informação interessante é que a distribuição da receita é bastante assimétrica centrada próximo de 0 e com a presença de vários outliers, indicando que a grande maioria dos filmes possui retornos que não costumam ultrapassar 50 milhões de lucro ou até nem chegam a ter lucro com algumas exceções.